In [4]:
import numpy as np
import pandas as pd
import webhoseio
from aylienapiclient import textapi
import re
from urllib.parse import urlparse

In [5]:
# Ashwin's key - we only get 1000 free requests a month, so be cautious with the number of times you perform queries
webhoseio.config(token="456af32b-3c58-455d-b9f8-90875bfc8f58") 

In [6]:
client = textapi.Client("4657dfa6", "bbfd5faa23a1020d92e946dd83bce6b7")

In [18]:
output_ex = webhoseio.query("filterWebData", {"q":"thread.title:(Uber isn't sure if it can 'remain a viable business' without building self-driving cars) site:businessinsider.com"})

In [20]:
output_ex

{'moreResultsAvailable': 0,
 'next': '/search?token=456af32b-3c58-455d-b9f8-90875bfc8f58&format=json&ts=1493425322135&q=thread.title%3A%28Uber+isn%27t+sure+if+it+can+%27remain+a+viable+business%27+without+building+self-driving+cars%29+site%3Abusinessinsider.com',
 'posts': [],
 'requestsLeft': 994,
 'totalResults': 0}

## Get the title and site_url of an article given a url

In [7]:
def getTitleAndSiteUrl(url):
    extract = client.Extract({"url": url, "best_image": False})
    title = extract['title']
    site_url = re.sub('www.', '', urlparse(url).hostname)
    
    return title, site_url

## Create the dataframe with the 20 features that are in the Kaggle Data Set. Also create the function that will add rows to the dataframe

In [8]:
# all the columns from the Kaggle data set
cols = ["uuid", "ord_in_thread", "author", "published", "title", "text", "language", "crawled", "site_url", "country", "domain_rank", "thread_title", "spam_score", "main_img_url", "replies_count", "participants_count", "likes", "comments", "shares", "type"]
df = pd.DataFrame(columns = cols)
df

,uuid,ord_in_thread,author,published,title,text,language,crawled,site_url,country,domain_rank,thread_title,spam_score,main_img_url,replies_count,participants_count,likes,comments,shares,type


In [16]:
def addRow(df, site_url, title):
    output = webhoseio.query("filterWebData", {"q":"thread.title:(" + title + ") site:" + site_url})
    post = output["posts"]
    print(output)
    thread = post["thread"]
    social = thread["social"]["facebook"]
    
    uuid = post["uuid"]
    ord_in_thread = post["ord_in_thread"]
    author = post["author"]
    published = post["published"]
    title = post["title"]
    text = post["text"]
    language = post["language"]
    crawled = post["crawled"]
    site_url = thread["site"]
    country = thread["country"]
    domain_rank = thread["domain_rank"]
    thread_title = thread["title"]
    spam_score = thread["spam_score"]
    img = thread["main_image"]
    replies = thread["replies_count"]
    participants = thread["participants_count"]
    likes = social["likes"]
    comments = social["comments"]
    shares = social["shares"]
    
    #data = [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0,0,0,0,0,0,0,0,0,0,0]]
    data = [[uuid, ord_in_thread, author, published, title, text, language, crawled, site_url, country, domain_rank, thread_title, spam_score, img, replies, participants, likes, comments, shares, "N/A"]]
    df_row = pd.DataFrame(data, columns=cols)
    #print(df_row)
    df = df.append(df_row, ignore_index=True)
    return df

In [17]:
title, site_url = getTitleAndSiteUrl("http://www.businessinsider.com/uber-questions-future-of-being-a-viable-business-without-self-driving-cars-2017-4")
print(title)
print(site_url)

df = addRow(df, site_url, title)
df

Uber isn't sure if it can 'remain a viable business' without building self-driving cars
businessinsider.com
{'posts': [], 'totalResults': 0, 'moreResultsAvailable': 0, 'requestsLeft': 995, 'next': '/search?token=456af32b-3c58-455d-b9f8-90875bfc8f58&format=json&ts=1493412057401&q=thread.title%3A%28Uber+isn%27t+sure+if+it+can+%27remain+a+viable+business%27+without+building+self-driving+cars%29+site%3Abusinessinsider.com'}


TypeError: list indices must be integers, not str